## Libraries

In [1]:
%%capture
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
plt.ion() 
import numpy as np
import pathlib 
import mne
from mne import Epochs 
from mne.time_frequency import psd_array_multitaper
import mplcursors 
import scipy
from datetime import datetime
import pandas as pd
from matplotlib.widgets import Cursor
from scipy.signal import welch
import re
import os
import glob
import csv
matplotlib.use('Qt5Agg')
pd.set_option('display.float_format', '{:.0f}'.format)
from mne.preprocessing import ICA
from mne.io.constants import FIFF

## Functions

### 1) Import fif files 

In [2]:
def import_sorted_fif_files(directory):
    """
    This function lists all .fif files in a specified directory, sorts them 
    based on the numeric value after the 'P' in the filename, and imports the 
    files into a list using MNE's read_raw_fif function.

    Parameters:
    directory (str): The path to the directory containing the .fif files.

    Returns:
    fif_tasks (list): A list of MNE Raw objects, each representing an imported 
    .fif file, in the order sorted by the numeric part after 'P' in the filename.
    """
    
    # List all files in the specified directory and filter for those ending with '.fif'
    fif_files = [f for f in os.listdir(directory) if f.endswith('.fif')]

    # Function to extract the numeric part after 'P' in the filename for sorting
    def extract_number(filename):
        """
        Extracts the number that follows 'P' and precedes '_T' in the filename.
        If no match is found, returns infinity to ensure that such files are sorted last.

        Parameters:
        filename (str): The filename to extract the number from.

        Returns:
        int or float: The extracted number or infinity if no match is found.
        """
        match = re.search(r'P(\d+)_T\d+\.fif', filename)
        return int(match.group(1)) if match else float('inf')
    
    # Sort the files by the numeric value extracted from the filename
    fif_files.sort(key=extract_number)

    # Import the .fif files in the sorted order and store them in a list
    fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]

    return fif_tasks


### 2) Get the UNIX start and end time for a .fif file

In [3]:
from datetime import datetime

def get_unix_times(fif_file):
    """
    Extracts the UNIX start and end time of an EEG recording from a .fif file.

    Parameters:
    ----------
    fif_file : mne.io.Raw
        A cleaned EEG signal stored in MNE's .fif format.

    Returns:
    -------
    eeg_start_unix : int
        The UNIX timestamp representing the start of the EEG recording, adjusted for the GMT to IST time offset (5 hours 30 minutes).
    
    eeg_end_unix : int
        The UNIX timestamp representing the end of the EEG recording, calculated using the signal duration and adjusted start time.
    """
    
    # Get the measurement start time from the .fif file metadata (meas_date).
    meas_date = fif_file.info['meas_date']
    
    # Handle different types of meas_date:
    # - If it's a tuple, take the first element and convert it to a timestamp.
    # - If it's already a datetime object, use its timestamp.
    # - If neither, raise a ValueError as the format is unrecognized.
    if isinstance(meas_date, tuple):
        meas_date_timestamp = meas_date[0].timestamp()  # meas_date[0] is a datetime object
    elif isinstance(meas_date, datetime):
        meas_date_timestamp = meas_date.timestamp()
    else:
        raise ValueError("meas_date is not in a recognized format. Expected a tuple or datetime object.")

    # Time correction factor to adjust for the GMT to IST (Indian Standard Time) offset, which is 5 hours and 30 minutes (19800 seconds).
    time_correction = 19800  # 5 hours 30 minutes = 5 * 3600 + 30 * 60 = 19800 seconds
    eeg_start_unix = int(meas_date_timestamp - time_correction)

    # Calculate the duration of the signal in seconds:
    # - sfreq: Sampling frequency (Hz) indicates how many data points are recorded per second.
    # - fif_file.times: An array representing time points of the recording.
    # The total duration is the length of the time array divided by the sampling frequency.
    sfreq = fif_file.info['sfreq']
    signal_duration_secs = int(len(fif_file.times) / sfreq)

    # Compute the UNIX timestamp for the end of the EEG recording by adding the signal duration to the start time.
    eeg_end_unix = int(eeg_start_unix + signal_duration_secs)

    return eeg_start_unix, eeg_end_unix


### 3) Import segment timestamps from .csv file

In [4]:
def import_segments(directory):
    """
    Imports segment start and end times from a CSV file and stores them in two separate lists.

    Parameters:
    ----------
    directory : str
        The file path to the CSV file containing segment start and end times.

    Returns:
    -------
    seg_start_time : list of str
        A list containing the start times of the segments, read from the first column of the CSV.
    
    seg_end_time : list of str
        A list containing the end times of the segments, read from the second column of the CSV.
    
    Notes:
    -----
    - The CSV file is expected to have at least two columns where:
        - The first column contains segment start times.
        - The second column contains segment end times.
    - The first row of the CSV is assumed to be a header and is skipped during import.
    """

    # Initialize the lists to store segment start and end times
    seg_start_time = []
    seg_end_time = []

    # Open the CSV file in read mode
    with open(directory, mode='r') as file:
        csv_reader = csv.reader(file)
        
        # Skip the header row (assumed to be the first row)
        next(csv_reader)
        
        # Iterate through the remaining rows in the CSV
        for row in csv_reader:
            # Append the start time from the first column to seg_start_time
            seg_start_time.append(row[0])  # First column: Start times
            
            # Append the end time from the second column to seg_end_time
            seg_end_time.append(row[1])    # Second column: End times

    # Optionally print the arrays to verify their contents
    print("Segment Start Times:", seg_start_time)
    print("Segment End Times:", seg_end_time)

    # Return the start and end times as two separate lists
    return seg_start_time, seg_end_time


### 4) Make relative start time array from .csv file

In [5]:
def make_time_array(fif_file, segment_csv_path):
    """
    Processes segment start and end times from a CSV file, aligns them with the 
    UNIX start time of an EEG recording in a .fif file, and calculates the relative 
    start times and other time-related metrics.

    Parameters:
    ----------
    fif_file : mne.io.Raw
        The EEG recording stored in MNE's .fif format.

    segment_csv_path : str
        The path to the CSV file containing the segment start and end times.

    Returns:
    -------
    rel_time_sec : np.array
        An array of relative segment start times, measured from the first segment's start time, in seconds.
    
    seg_start_array : list of int
        A list of absolute segment start times in UNIX time, extracted and converted from the CSV.
    
    seg_end_array : list of int
        A list of absolute segment end times in UNIX time, extracted and converted from the CSV.
    
    t_seg_min : int
        The offset of the first segment start time relative to the beginning of the EEG recording (in seconds).
    
    t_seg_max : int
        The offset of the last segment end time relative to the beginning of the EEG recording (in seconds).
    
    Notes:
    -----
    - The function prints time-related details of the EEG recording and segments.
    - Assumes that the first segment's start time is the reference point for calculating relative times.
    """

    # Import segment start and end times from the CSV file
    seg_start, seg_end = import_segments(segment_csv_path)
    
    # Count the number of segments
    segment_count = len(seg_start)
    
    # Get the UNIX start and end time of the EEG recording
    unix_start_reading, unix_end_reading = get_unix_times(fif_file)
    
    # Calculate the total duration of the EEG recording in seconds
    total_reading_time = unix_end_reading - unix_start_reading
    
    # Convert segment start and end times from strings to integers
    seg_start_array = [int(m) for m in seg_start]
    seg_end_array = [int(m) for m in seg_end]
    
    # Use the first segment's start time as the reference point
    seg_start = seg_start_array[0]
    
    # Calculate relative start times for each segment, using the first segment's start time as the base
    seg_start_array_rel = [x - seg_start for x in seg_start_array]
    rel_time_sec = np.array(seg_start_array_rel)

    # Calculate the total duration of all segments
    total_segment_duration = seg_end_array[-1] - seg_start_array[0]
    
    # Calculate the offset of the first segment's start time relative to the start of the EEG recording
    offset = seg_start_array[0] - unix_start_reading
    
    # Calculate the minimum and maximum relative times for the segments
    t_seg_min = offset
    t_seg_max = offset + total_segment_duration
    
    # Print the time-related details of the EEG reading and segments
    print(f'Unix start time of the reading = {unix_start_reading}')
    print(f'Unix end time of the reading = {unix_end_reading}')
    print(f'Duration of signal = {total_reading_time} sec')
    print(f'No of segments: {segment_count}')
    print(f'Total duration of segments = {total_segment_duration}')
    print(f'Relative segment start times in seconds: {rel_time_sec}')
    print(f't_seg_min = {t_seg_min}')
    print(f't_seg_max = {t_seg_max}')
    print(f'Segments start at {t_seg_min} seconds and end at {t_seg_max} seconds')

    # Return the calculated values
    return rel_time_sec, seg_start_array, seg_end_array, t_seg_min, t_seg_max


### 5) Cropping segments - using relative segment start times

In [6]:
def crop_segments(fif_file, rel_time_sec, t_seg_max):
    """
    Crops a given .fif file based on provided time segments and stores these cropped segments into a list.

    Parameters:
    -----------
    fif_file : mne.io.Raw
        Cleaned EEG reading to be segmented. This is typically one of the readings selected for protocol analysis.
    
    rel_time_sec : np.ndarray
        An array of relative start times for the segments (in seconds), where each value represents the time 
        from the start of the signal. The length of this array should be equal to the number of segments + 1.

    t_seg_max : int
        The maximum time (in seconds) to which the .fif file should be cropped.

    Returns:
    --------
    cropped_fif_segments : list of mne.io.Raw
        A list containing the cropped .fif segments, where each element is a cropped version of the original file.
    
    segmentwise_events : list of np.ndarray
        A list of event arrays generated for each segment, where each array contains events that are spaced at 
        a fixed interval (1-second duration).
    
    Notes:
    ------
    - This function assumes the sampling frequency is 256 Hz.
    - The .fif file is cropped at the start and end times specified by `rel_time_sec`.
    - An additional crop is performed for the final segment from the last relative time to the full file duration.
    """

    # Calculate the number of segments (subtract 1 since rel_time_sec has n+1 elements)
    segment_count = len(rel_time_sec) - 1
    
    # Create a copy of the fif_file to preserve the original data
    fif_temp_last = fif_file.copy()

    # Calculate the total duration of the .fif file in seconds, assuming a sampling frequency of 256 Hz
    fif_file_duration = int(len(fif_file.times) / 256)
    print(f'Duration of .fif file: {fif_file_duration} seconds')

    # Initialize lists to store the cropped .fif segments and segment-wise event arrays
    cropped_fif_segments = []
    segmentwise_events = []

    # Loop through each segment to crop the .fif file and generate events
    for i in range(segment_count):
        x = rel_time_sec[i]
        y = rel_time_sec[i + 1]
        print('-----')
        print(f'Segment {i} start time = {x} seconds')
        print(f'Segment {i} end time = {y} seconds')
        print('-----')

        # Copy the original .fif file and crop it based on the current segment's start and end times
        fif_temp = fif_file.copy()
        fif_temp_cropped = fif_temp.crop(tmin=x, tmax=y)
        
        # Create fixed-length events (duration 1 second) for the current segment
        seg_event = mne.make_fixed_length_events(fif_file, start=x, stop=y, duration=1.0)
        
        # Append the cropped segment and the events array to their respective lists
        cropped_fif_segments.append(fif_temp_cropped)
        segmentwise_events.append(seg_event)
        print(f'Segment {i} appended to cropped segments')
        print(f'Events array for segment {i} appended')

    # Handle the final segment, which extends from the last relative start time to the end of the .fif file
    fif_temp_cropped_last = fif_temp_last.crop(tmin=rel_time_sec[-1], tmax=fif_file_duration)
    cropped_fif_segments.append(fif_temp_cropped_last)
    
    # Create events for the last segment and append them to the list
    seg_event = mne.make_fixed_length_events(fif_file, start=rel_time_sec[-1], stop=fif_file_duration, duration=1.0)
    segmentwise_events.append(seg_event)
    print(f'Last segment appended to cropped segments')
    print(f'Events array for the last segment appended')

    # Return the list of cropped segments and the corresponding event arrays
    return cropped_fif_segments, segmentwise_events


### 6) Crop and copy using UNIX start and end time

In [7]:
def crop_copy(raw, start_unix, end_unix, t1, t2):
    """
    Crop a raw MNE object based on a time window specified by UNIX timestamps.

    This function takes a raw EEG/MEG recording, represented as an `mne.io.Raw` object, and crops it based on
    two provided UNIX timestamps (t1 and t2), which specify the start and end of the desired time window. 
    The timestamps are converted to relative times based on the start of the recording.

    Parameters:
    -----------
    raw : mne.io.Raw
        The raw MNE object containing the recording to be cropped. This object holds EEG or MEG data, along with
        associated metadata.
    
    start_unix : int
        The UNIX timestamp representing the start time of the recording. This is the time when the EEG/MEG recording
        began, and it is used to calculate the relative crop times.
    
    end_unix : int
        The UNIX timestamp representing the end time of the recording. This is included for completeness but is not 
        used in the cropping calculation. 
    
    t1 : int
        The seconds timestamp indicating the start of the desired crop window. This is the time at which the cropped 
        segment should begin.
    
    t2 : int
        The seconds timestamp indicating the end of the desired crop window. This is the time at which the cropped 
        segment should end.

    Returns:
    --------
    raw_temp : mne.io.Raw
        A new `mne.io.Raw` object that contains the cropped data. This object is a copy of the original `raw` object, 
        but restricted to the time window specified by `t1` and `t2`.

    Notes:
    ------
    - The cropping is based on relative times (in seconds) calculated from the start of the recording (start_unix).
    - If `t1` is earlier than the recording start, the crop window will begin from 0 seconds (the start of the recording).
    - If `t2` is later than the recording end, the crop window will extend only to the end of the available data.
    - This function returns a copy of the original `raw` object; the original remains unmodified.
    """

    # Convert the recording start time from UNIX timestamp to a datetime object
    recording_start_time = datetime.utcfromtimestamp(start_unix)

    # Convert UNIX timestamps t1 and t2 to datetime objects (start and end of the crop window)
    start_time = datetime.utcfromtimestamp(t1)
    end_time = datetime.utcfromtimestamp(t2)

    # Calculate the relative times (in seconds) from the start of the recording
    relative_start_time = (start_time - recording_start_time).total_seconds()
    relative_end_time = (end_time - recording_start_time).total_seconds()

    # Ensure the relative start time is not before the start of the recording
    relative_start_time = max(0, relative_start_time)
    
    # Ensure the relative end time does not exceed the length of the recording
    relative_end_time = min(raw.times[-1], relative_end_time)

    # Print out the calculated relative times for debugging
    print(f'Relative start time (seconds): {relative_start_time}')
    print(f'Relative end time (seconds): {relative_end_time}')

    # Copy the raw object to create a new instance for cropping
    raw_temp = raw.copy()

    # Crop the copied raw object using the calculated relative times
    raw_temp.crop(tmin=relative_start_time, tmax=relative_end_time)

    return raw_temp


### 7) Equal length epochs for EEG signals

In [8]:
def epoch_equally(rawf_8,win_len):
    """
    Creates equal length epochs for the given raw object.
    The length of the epochs is equal to the sampling frequency, hence it creates epochs of 1 second each.

    Parameters :
    rawf_8 (raw) : Raw object containing 8 channels of EEG data
    emotive_montage_8 (montage) : Custom montage object containing diigitization points for frontal 8 electrodes 

    Returns :
    epochs_8 (epochs): Equally epoched raw object
    """
    mne.set_log_level('debug') 

    ### Montage selection -------------------------------------------------------------------------------------------
    # Dictionary mapping channel names to (x, y, z) coordinates

    positions_8 = { 
        'AF3': (-0.0337007,  0.0768371,  0.021227),
        'F7': (-0.0702629,  0.0424743, -0.01142),
        'F3': (-0.0502438,  0.0531112,  0.042192),
        'FC5': (-0.0772149,  0.0186433,  0.02446),
        'FC6': (0.0795341, 0.0199357, 0.024438),
        'F4': (0.0518362, 0.0543048, 0.040814 ),
        'F8': (0.0730431,  0.0444217, -0.012),
        'AF4': (0.0357123, 0.0777259, 0.021956),
    }


    # Create a customised DigMontage object
    emotiv_montage_8 = mne.channels.make_dig_montage(ch_pos= positions_8, coord_frame='head')

    reject_criteria = dict(eeg=200e-6)  # Additional amplitude-based criteria

    # Define the length of each epoch in samples (e.g., 256 samples/second * 5 = 5 secoonds of epoch)
    epoch_length = rawf_8.info['sfreq'] * win_len 
    sfreq = rawf_8.info['sfreq']

    tmax = (epoch_length-1) / sfreq

    # Create events based on a fixed-length sliding window
    events = np.array([[i, 0, 1] for i in range(0, len(rawf_8.times), int(epoch_length))])
    #mne.viz.plot_events(events, sfreq=rawf_8.info['sfreq'])
    print('--------------------------------------------------')
    print('--------------------------------------------------')
    print(events[:10])
    print('--------------------------------------------------')
    print('--------------------------------------------------')

    print(f"Raw data starts at {rawf_8.times[0]} and ends at {rawf_8.times[-1]}")

    print('--------------------------------------------------')
    print('--------------------------------------------------')


    epochs_8 = mne.Epochs(rawf_8, events, tmin=0, tmax=tmax,baseline=None, detrend=1, reject = reject_criteria, reject_by_annotation= True, preload=True)

    #Apply the custom montage to create the topomaps
    epochs_8.set_montage(emotiv_montage_8)

    return epochs_8


### 8) Baseline Correction

In [13]:
def resting_bc(rawf_8, win_len):
    """
    Perform bandpower computation on EEG data during a resting-state condition.

    This function takes a raw EEG object (`rawf_8`), crops the data to a specific time range representing a 
    resting-state condition, and then epochs the data into equally sized time windows. The function computes 
    relative bandpowers for different frequency bands within the resting-state period.

    Parameters:
    -----------
    rawf_8 : mne.io.Raw
        A raw EEG object containing 8 channels of EEG data, typically from frontal electrodes.
    
    win_len : float
        The length of each epoch window in seconds. This determines how long each time segment (epoch) will be for the 
        bandpower calculations.

    Returns:
    --------
    rest_dp : float
        Average delta band power (1-4 Hz) computed from the resting-state epochs.
    
    rest_tp : float
        Average theta band power (4-8 Hz) computed from the resting-state epochs.
    
    rest_ap : float
        Average alpha band power (8-12 Hz) computed from the resting-state epochs.
    
    rest_bp : float
        Average beta band power (12-30 Hz) computed from the resting-state epochs.

    Notes:
    ------
    - The time window for the resting-state data is predefined, with a start time of 0.0 seconds (`st_rest`) 
      and an end time of 30.0 seconds (`et_rest`).
    - This function uses the `epoch_equally` function to segment the cropped data into epochs of equal length.
    - The bandpowers are computed using the `average_bandpowers_relative` function, which calculates the 
      relative bandpower for delta, theta, alpha, and beta frequency bands.
    """

    # Define the names of the 8 channels typically used for frontal EEG readings
    channels_8 = ['AF3', 'F7', 'F3', 'FC5', 'FC6', 'F4', 'F8', 'AF4']

    # Define the start and end times for the resting-state condition (in seconds)
    st_rest = 0.0  # Start time for resting state (can be adjusted)
    et_rest = 30.0  # End time for resting state (can be adjusted)

    # Crop the raw EEG object to the specified time range for the resting state
    rest_raw = rawf_8.copy().crop(tmin=st_rest, tmax=et_rest)

    # Epoch the cropped signal into equal-length epochs using the specified window length
    rest_epochs = epoch_equally(rest_raw, win_len)

    # Compute the average relative band powers for delta, theta, alpha, and beta bands
    rest_dp, rest_tp, rest_ap, rest_bp = average_bandpowers(rest_epochs)

    # Return the computed band powers
    return rest_dp, rest_tp, rest_ap, rest_bp


### 9) Bandpower Calculation - for single channel EEG Signal

In [10]:
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.
    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch' or 'multitaper'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    freq_res : float
      Frequency resolution of the bandpower calculation
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', n_jobs=1, remove_dc= True, max_iter = 1024, verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp, freq_res

In [11]:
def totalpower(data, sf):
    """
    Calculate the total power of a signal by computing its power spectral density (PSD) 
    using the multitaper method and integrating the PSD over all frequencies.

    Parameters
    ----------
    data : array_like
        Time series data of the signal. Typically, this should be a 1D array representing 
        the signal values over time.
    sf : float
        Sampling frequency of the data, in Hz. This value is required for accurate 
        frequency domain representation.

    Returns
    -------
    tp : float
        The total power of the signal, obtained by integrating the PSD over the frequency 
        range of interest. This represents the area under the PSD curve.

    Notes
    -----
    - This function uses the multitaper method to compute the PSD, which provides a robust 
      spectral estimation technique that reduces spectral leakage.
    - The total power is computed using Simpson's rule for integration, which provides an 
      accurate approximation by fitting parabolas to the data segments.

    Examples
    --------
    >>> import numpy as np
    >>> sf = 1000  # Sampling frequency in Hz
    >>> data = np.random.randn(1000)  # Simulated signal
    >>> total_power = totalpower(data, sf)
    >>> print("Total power:", total_power)

    """
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    # Compute PSD using multitaper method
    psd, freqs = psd_array_multitaper(data, sf, adaptive=True, normalization='full', 
                                      n_jobs=1, remove_dc=True, max_iter=1024, verbose=0)

    # Frequency resolution, i.e., the distance between frequency bins
    freq_res = freqs[1] - freqs[0]

    # Calculate total power by integrating the PSD over all frequencies using Simpson's rule
    tp = simps(psd, dx=freq_res)
    return tp

In [12]:
def sum_total_channel_power(sketching_signal_fif):
    # List to store total power for each channel
    total_power_per_channel = []
    sf = 256

    temp_data = sketching_signal_fif.get_data()
    
    # Calculate total power for each channel
    for channel_data in temp_data:
        total_power = totalpower(channel_data, sf)
        total_power_per_channel.append(total_power)

    sum_total_power = np.sum(total_power_per_channel)

    return sum_total_power

### 10) Average Bandpower Across All Channels - Absolute

In [14]:
def average_bandpowers(epochs_8):
    band_1 = [1,4]
    band_2 = [4,8]
    band_3 = [8,13]
    band_4 = [13,30]
    
    epochs_data = epochs_8.get_data()
    n_epochs, n_chs, n_samples = epochs_data.shape

    # Initialize arrays to store total power for each epoch
    total_delta_power = np.zeros(n_epochs)
    total_theta_power = np.zeros(n_epochs)
    total_alpha_power = np.zeros(n_epochs)
    total_beta_power = np.zeros(n_epochs)

    # Sampling frequency
    sfreq = epochs_8.info['sfreq']

    # Loop through each epoch and channel to compute band powers
    for x in range(n_epochs):
        # Reset band power sums for the current epoch
        total_bp_delta = 0
        total_bp_theta = 0
        total_bp_alpha = 0
        total_bp_beta = 0

        for y in range(n_chs):
            # Sum the band power for each channel within the epoch
            bp_delta, _ = bandpower(epochs_data[x][y], sfreq, band_1, 'multitaper', relative=False)
            bp_theta, _ = bandpower(epochs_data[x][y], sfreq, band_2, 'multitaper', relative=False)
            bp_alpha, _ = bandpower(epochs_data[x][y], sfreq, band_3, 'multitaper', relative=False)
            bp_beta, _ = bandpower(epochs_data[x][y], sfreq, band_4, 'multitaper', relative=False)

            # Accumulate the total bandpower across channels for this epoch
            total_bp_delta += bp_delta
            total_bp_theta += bp_theta
            total_bp_alpha += bp_alpha
            total_bp_beta += bp_beta

        # Store the total power for this epoch
        total_delta_power[x] = total_bp_delta
        total_theta_power[x] = total_bp_theta
        total_alpha_power[x] = total_bp_alpha
        total_beta_power[x] = total_bp_beta

    # Compute final mean power across all epochs for each frequency band
    final_mean_delta = np.mean(total_delta_power)
    final_mean_theta = np.mean(total_theta_power)
    final_mean_alpha = np.mean(total_alpha_power)
    final_mean_beta = np.mean(total_beta_power)

    # Output the final means for each band
    print("Final Mean Delta Power across epochs:", final_mean_delta)
    print("Final Mean Theta Power across epochs:", final_mean_theta)
    print("Final Mean Alpha Power across epochs:", final_mean_alpha)
    print("Final Mean Beta Power across epochs:", final_mean_beta)

    return final_mean_delta,final_mean_theta,final_mean_alpha,final_mean_beta

# Main Code

### Setup 1 : Segment wise calculations

#### Import files from protocol analysis

In [ ]:
#Import details for fif files for task 3
directory_fif = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3'
fif_task_3 = import_sorted_fif_files(directory_fif)

#Select the files to be processed 
fif_protocol = {'P7_T3': fif_task_3[5],
                'P8_T3': fif_task_3[6],
                'P9_T3': fif_task_3[7],
                'P22_T3': fif_task_3[20],
                'P24_T3': fif_task_3[22],
                'P27_T3': fif_task_3[25],
                'P30_T3': fif_task_3[28]}

#### Segment Calculations

##### Get segment time stamps

In [ ]:
#Select the file from protocol list
fif_pro = fif_protocol['P30_T3'].copy()

# Define the file path to the segment CSV file
segment_csv_path = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/PSD Calculations/Segment Wise PSD/Segment Windows/Segment_Time_Stamps - P30 .csv'

In [ ]:
fif_pro.times[-1]

In [ ]:
#Get time details for segments
rel_time_sec, seg_start_time, seg_end_time,t_seg_min,t_seg_max = make_time_array(fif_pro, segment_csv_path)

# reading_start_time,reading_end_time = get_unix_times(fif_pro)

# print(f'Total duration of segments = {total_segment_duration}')
# offset = seg_start_time[0] - reading_start_time
# t_seg_min = offset
# t_seg_max = offset + total_segment_duration
# print(f'Segments start at {t_seg_min} second and end at {t_seg_max} second')

In [ ]:
#Crop only the duration for which segments are present in the signal. Use this cropped file for segmentwise ABP calculation.
fif_temp = fif_pro.copy()
fif_all_segments = fif_temp.crop(tmin=t_seg_min, tmax=t_seg_max)
fif_all_segments.plot()

In [ ]:
cropped_fif_segments,segmentwise_events = crop_segments(fif_all_segments,rel_time_sec,t_seg_max)

#### Absolute Bandpowers

In [ ]:
cropped_fif_segments

In [ ]:
total_power_array_Pn = [0] * len(cropped_fif_segments)

In [ ]:
i = 0 
for seg in cropped_fif_segments :
    total_power_array_Pn[i] = sum_total_channel_power(seg)
    i = i+1    

In [ ]:
tp = pd.DataFrame(total_power_array_Pn)

In [ ]:
tp.to_csv('/Users/apple/Desktop/temp/temp.csv', index=False)

In [ ]:
# List to store total power for each channel
total_power_per_channel = []

# Calculate total power for each channel
for channel_data in data:
    total_power = totalpower(channel_data, sf)
    total_power_per_channel.append(total_power)

# Display total power for each channel
for i, power in enumerate(total_power_per_channel):
    print(f"Total power for channel {i+1}: {power}")

In [ ]:
reject_criteria = dict(eeg=200e-6)
i = 0
delta_abp = np.zeros((len(cropped_fif_segments)))
theta_abp = np.zeros((len(cropped_fif_segments)))
alpha_abp = np.zeros((len(cropped_fif_segments)))
beta_abp = np.zeros((len(cropped_fif_segments)))
total_power = np.zeros((len(cropped_fif_segments)))

for seg in cropped_fif_segments :
    seg_temp_fif = cropped_fif_segments[i].copy()
    epochs_new = mne.Epochs(seg_temp_fif, segmentwise_events[i], event_id=None, tmin=0.0, tmax=1.0, baseline=None, reject=reject_criteria ,reject_by_annotation= True, preload=True)
    seg_mean_delta,seg_mean_theta,seg_mean_alpha,seg_mean_beta = average_bandpowers_absolute(epochs_new)
    delta_abp[i] = seg_mean_delta
    theta_abp[i] = seg_mean_theta
    alpha_abp[i] = seg_mean_alpha
    beta_abp[i] = seg_mean_beta
    i = i+1

abp_values = np.column_stack((delta_abp, theta_abp, alpha_abp,beta_abp))
abp_values_df = pd.DataFrame(abp_values, columns=['Segment delta power', 'Segment theta power', 'Segment alpha power', 'Segment beta power'])
pd.options.display.float_format ='{:.2e}'.format
abp_values_df.attrs['title'] = 'ABP for all segments of P24_T3'
abp_values_df



In [ ]:
abp_values_df.to_csv('/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/PSD Calculations/Segment Wise PSD/Segmentwise ABP/P24_T3_Absolute.csv', index=True)

#### Relative Bandpowers

In [ ]:
reject_criteria = dict(eeg=200e-6)
i = 0
delta_abp = np.zeros((len(cropped_fif_segments)))
theta_abp = np.zeros((len(cropped_fif_segments)))
alpha_abp = np.zeros((len(cropped_fif_segments)))
beta_abp = np.zeros((len(cropped_fif_segments)))

for seg in cropped_fif_segments :
    seg_temp_fif = cropped_fif_segments[i].copy()
    epochs_new = mne.Epochs(seg_temp_fif, segmentwise_events[i], event_id=None, tmin=0.0, tmax=1.0, baseline=None, reject=reject_criteria ,reject_by_annotation= True, preload=True)
    seg_mean_delta,seg_mean_theta,seg_mean_alpha,seg_mean_beta = average_bandpowers_relative(epochs_new)
    delta_abp[i] = seg_mean_delta
    theta_abp[i] = seg_mean_theta
    alpha_abp[i] = seg_mean_alpha
    beta_abp[i] = seg_mean_beta
    i = i+1

abp_values = np.column_stack((delta_abp, theta_abp, alpha_abp,beta_abp))
abp_values_df = pd.DataFrame(abp_values, columns=['Segment delta power', 'Segment theta power', 'Segment alpha power', 'Segment beta power'])
pd.options.display.float_format ='{:.2e}'.format
abp_values_df.attrs['title'] = 'Relative ABP for all segments of P24_T3'
abp_values_df


In [ ]:
abp_values_df.to_csv('/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/PSD Calculations/Segment Wise PSD/Segmentwise ABP/P24_T3_Relative.csv', index=True)

####

####

####

### Setup 2 : Full signal calculations

In [15]:
#Import details for fif files for all tasks
directory_fif_task1 = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1'
directory_fif_task2 = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 2'
directory_fif_task3 = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3'
fif_task_1 = import_sorted_fif_files(directory_fif_task1)
fif_task_2 = import_sorted_fif_files(directory_fif_task2)
fif_task_3 = import_sorted_fif_files(directory_fif_task3)

Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P1_T1.fif...
    Range : 0 ... 24831 =      0.000 ...    96.996 secs
Ready.


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P1_T1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]


Reading 0 ... 24831  =      0.000 ...    96.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P3_T1.fif...
    Range : 0 ... 22783 =      0.000 ...    88.996 secs
Ready.
Reading 0 ... 22783  =      0.000 ...    88.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P4_T1.fif...
    Range : 0 ... 20991 =      0.000 ...    81.996 secs
Ready.
Reading 0 ... 20991  =      0.000 ...    81.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P5_T1.fif...
    Range : 0 ... 27135 =      0.000 ...   105.996 secs
Ready.
Reading 0 ... 27135  =      0.000 ...   105.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P6_T1.fif...
    Range : 0 ... 27647 =      0.000 ...   107.996 secs
Ready.
Reading 0 ..

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P3_T1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]
/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P4_T1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks =

Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P8_T1.fif...
    Range : 0 ... 18943 =      0.000 ...    73.996 secs
Ready.
Reading 0 ... 18943  =      0.000 ...    73.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P9_T1.fif...
    Range : 0 ... 16383 =      0.000 ...    63.996 secs
Ready.
Reading 0 ... 16383  =      0.000 ...    63.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P10_T1.fif...
    Range : 0 ... 22271 =      0.000 ...    86.996 secs
Ready.
Reading 0 ... 22271  =      0.000 ...    86.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P11_T1.fif...
    Range : 0 ... 17919 =      0.000 ...    69.996 secs
Ready.
Reading 0 ... 17919  =      0.000 ...    69.996 secs...
Opening ra

...

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P23_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]
/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P24_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks

Ready.
Reading 0 ... 189439  =      0.000 ...   739.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P28_T3.fif...
    Range : 0 ... 239359 =      0.000 ...   934.996 secs
Ready.
Reading 0 ... 239359  =      0.000 ...   934.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P29_T3.fif...
    Range : 0 ... 135167 =      0.000 ...   527.996 secs
Ready.
Reading 0 ... 135167  =      0.000 ...   527.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P30_T3.fif...
    Range : 0 ... 236543 =      0.000 ...   923.996 secs
Ready.
Reading 0 ... 236543  =      0.000 ...   923.996 secs...


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P28_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]
/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P29_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks

Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P31_T3.fif...
    Range : 0 ... 89343 =      0.000 ...   348.996 secs
Ready.
Reading 0 ... 89343  =      0.000 ...   348.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P32_T3.fif...
    Range : 0 ... 70143 =      0.000 ...   273.996 secs
Ready.
Reading 0 ... 70143  =      0.000 ...   273.996 secs...


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P31_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]
/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P32_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks

In [16]:
participant_list = ['P1',
 'P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9',
'P10',                   
'P11',
'P12',
'P13',
'P14',
'P15',
'P16',
'P17',
'P18',
'P19',
'P20',
'P21',
'P22',
'P23',
'P24',
'P25',
'P26',
'P27',
'P28',
'P29',
'P30',
'P31',
'P32']

#### Baseline Calculations

In [23]:
# Calculating average band power for delta,theta,alpha and beta for the first 30 seconds.

i = 0
j = 0 
k = 0

window_length_bc = 5

#Absolute Baseline Bandpower Arrays
task_1_rest_d = np.zeros((len(fif_task_1)))
task_1_rest_t = np.zeros((len(fif_task_1)))
task_1_rest_a = np.zeros((len(fif_task_1)))
task_1_rest_b = np.zeros((len(fif_task_1))) 
task_2_rest_d = np.zeros((len(fif_task_2)))
task_2_rest_t = np.zeros((len(fif_task_2)))
task_2_rest_a = np.zeros((len(fif_task_2)))
task_2_rest_b = np.zeros((len(fif_task_2)))
task_3_rest_d = np.zeros((len(fif_task_3)))
task_3_rest_t = np.zeros((len(fif_task_3)))
task_3_rest_a = np.zeros((len(fif_task_3)))
task_3_rest_b = np.zeros((len(fif_task_3)))

#Relative Baseline Bandpower Arrays
rtask_1_rest_d = np.zeros((len(fif_task_1)))
rtask_1_rest_t = np.zeros((len(fif_task_1)))
rtask_1_rest_a = np.zeros((len(fif_task_1)))
rtask_1_rest_b = np.zeros((len(fif_task_1))) 
rtask_2_rest_d = np.zeros((len(fif_task_2)))
rtask_2_rest_t = np.zeros((len(fif_task_2)))
rtask_2_rest_a = np.zeros((len(fif_task_2)))
rtask_2_rest_b = np.zeros((len(fif_task_2)))
rtask_3_rest_d = np.zeros((len(fif_task_3)))
rtask_3_rest_t = np.zeros((len(fif_task_3)))
rtask_3_rest_a = np.zeros((len(fif_task_3)))
rtask_3_rest_b = np.zeros((len(fif_task_3)))


for item in fif_task_1 :
    temp_fif = fif_task_1[i].copy()
    #Absolute
    rest_dp,rest_tp,rest_ap,rest_bp = resting_bc(temp_fif, win_len = window_length_bc)
    task_1_rest_d[i] = rest_dp
    task_1_rest_t[i] = rest_tp
    task_1_rest_a[i] = rest_ap
    task_1_rest_b[i] = rest_bp
    # #Relative
    # rrest_dp,rrest_tp,rrest_ap,rrest_bp = resting_bc_relative(temp_fif, win_len = window_length_bc)
    # rtask_1_rest_d[i] = rrest_dp
    # rtask_1_rest_t[i] = rrest_tp
    # rtask_1_rest_a[i] = rrest_ap
    # rtask_1_rest_b[i] = rrest_bp
    i = i+1

for item in fif_task_2 :
    temp_fif = fif_task_2[j].copy()
    #Absolute
    rest_dp,rest_tp,rest_ap,rest_bp = resting_bc(temp_fif, win_len = window_length_bc) 
    task_2_rest_d[j] = rest_dp
    task_2_rest_t[j] = rest_tp
    task_2_rest_a[j] = rest_ap
    task_2_rest_b[j] = rest_bp
    # #Relative
    # rrest_dp,rrest_tp,rrest_ap,rrest_bp = resting_bc_relative(temp_fif, win_len = window_length_bc)
    # rtask_2_rest_d[j] = rrest_dp
    # rtask_2_rest_t[j] = rrest_tp
    # rtask_2_rest_a[j] = rrest_ap
    # rtask_2_rest_b[j] = rrest_bp
    j = j+1

for item in fif_task_3 :
    temp_fif = fif_task_3[k].copy()
    #Absolute
    rest_dp,rest_tp,rest_ap,rest_bp = resting_bc(temp_fif, win_len = window_length_bc)
    task_3_rest_d[k] = rest_dp
    task_3_rest_t[k] = rest_tp
    task_3_rest_a[k] = rest_ap
    task_3_rest_b[k] = rest_bp    
    # #Relative
    # rrest_dp,rrest_tp,rrest_ap,rrest_bp = resting_bc_relative(temp_fif, win_len = window_length_bc)
    # rtask_3_rest_d[k] = rrest_dp
    # rtask_3_rest_t[k] = rrest_tp
    # rtask_3_rest_a[k] = rrest_ap
    # rtask_3_rest_b[k] = rrest_bp
    k = k+1

--------------------------------------------------
--------------------------------------------------
[[   0    0    1]
 [1280    0    1]
 [2560    0    1]
 [3840    0    1]
 [5120    0    1]
 [6400    0    1]
 [7680    0    1]]
--------------------------------------------------
--------------------------------------------------
Raw data starts at 0.0 and ends at 30.0
--------------------------------------------------
--------------------------------------------------
Not setting metadata
7 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 7 events and 1280 original time points ...
    Getting epoch for 0-1280
    Getting epoch for 1280-2560
    Getting epoch for 2560-3840
    Getting epoch for 3840-5120
    Getting epoch for 5120-6400
    Getting epoch for 6400-7680
    Getting epoch for 7680-8960
1 bad epochs dropped
Data is self data: False
Data is self data: True


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 1.8620907680536867e-10
Final Mean Theta Power across epochs: 9.806841809422662e-11
Final Mean Alpha Power across epochs: 3.410443174560435e-10
Final Mean Beta Power across epochs: 1.1525078832219161e-10
Cropping annotations 2023-11-03 14:48:46+00:00 - 2023-11-03 14:49:16.003906+00:00
  [0] Dropping (2023-11-03 14:49:21.580845+00:00 - 2023-11-03 14:49:22.329685+00:00: bad_signal)
  [1] Dropping (2023-11-03 14:49:27.210556+00:00 - 2023-11-03 14:49:29.039692+00:00: bad_signal)
Cropping complete (kept 0)
--------------------------------------------------
--------------------------------------------------
[[   0    0    1]
 [1280    0    1]
 [2560    0    1]
 [3840    0    1]
 [5120    0    1]
 [6400    0    1]
 [7680    0    1]]
--------------------------------------------------
--------------------------------------------------
Raw data starts at 0.0 and ends at 30.0
--------------------------------------------------
----------------------------------

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 1.8643586683527058e-10
Final Mean Theta Power across epochs: 1.3714113117186812e-10
Final Mean Alpha Power across epochs: 2.189335090029112e-10
Final Mean Beta Power across epochs: 1.5597516110167146e-10
--------------------------------------------------
--------------------------------------------------
[[   0    0    1]
 [1280    0    1]
 [2560    0    1]
 [3840    0    1]
 [5120    0    1]
 [6400    0    1]
 [7680    0    1]]
--------------------------------------------------
--------------------------------------------------
Raw data starts at 0.0 and ends at 30.0
--------------------------------------------------
--------------------------------------------------
Not setting metadata
7 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 7 events and 1280 original time points ...
    Getting epoch for 0-1280
    Getting epoch for 1280-2560
    Getting epoch for 2560-3840
    Gettin

...

Final Mean Delta Power across epochs: 1.4960597821405044e-10
Final Mean Theta Power across epochs: 6.657880153823443e-11
Final Mean Alpha Power across epochs: 1.4920388025846815e-10
Final Mean Beta Power across epochs: 1.0816815955449509e-10
Cropping annotations 2024-07-31 17:45:13+00:00 - 2024-07-31 17:45:43.003906+00:00
  [0] Dropping (2024-07-31 17:47:51.367462+00:00 - 2024-07-31 17:47:53.576050+00:00: bad_signal)
  [1] Dropping (2024-07-31 17:48:59.862411+00:00 - 2024-07-31 17:49:01.218917+00:00: bad_signal)
  [2] Dropping (2024-07-31 17:49:08.474579+00:00 - 2024-07-31 17:49:09.810639+00:00: bad_signal)
  [3] Dropping (2024-07-31 17:49:36.942871+00:00 - 2024-07-31 17:49:38.428894+00:00: bad_signal)
  [4] Dropping (2024-07-31 17:50:44.732452+00:00 - 2024-07-31 17:50:47.404571+00:00: bad_signal)
  [5] Dropping (2024-07-31 17:51:00.728210+00:00 - 2024-07-31 17:51:01.996093+00:00: bad_signal)
Cropping complete (kept 0)
--------------------------------------------------
----------------

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 1.3286139606614787e-10
Final Mean Theta Power across epochs: 8.537707300406414e-11
Final Mean Alpha Power across epochs: 9.913576327413856e-11
Final Mean Beta Power across epochs: 9.365779282370918e-11
Cropping annotations 2024-08-01 12:25:13+00:00 - 2024-08-01 12:25:43.003906+00:00
  [0] Keeping  (2024-08-01 12:25:13+00:00 - 2024-08-01 12:25:14.356510+00:00 -> 0.0 - 1.3565099239349365)
  [1] Dropping (2024-08-01 12:25:54.584312+00:00 - 2024-08-01 12:25:57.379131+00:00: bad_signal)
  [2] Dropping (2024-08-01 12:29:46.518921+00:00 - 2024-08-01 12:29:46.996094+00:00: bad_signal)
Cropping complete (kept 1)
--------------------------------------------------
--------------------------------------------------
[[   0    0    1]
 [1280    0    1]
 [2560    0    1]
 [3840    0    1]
 [5120    0    1]
 [6400    0    1]
 [7680    0    1]]
--------------------------------------------------
--------------------------------------------------
Raw data starts at 0

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 5.2643422761198596e-11
Final Mean Theta Power across epochs: 3.687706215770847e-11
Final Mean Alpha Power across epochs: 3.405510986287538e-11
Final Mean Beta Power across epochs: 6.266367956181664e-11


In [24]:
#Creating a dataframe object to export csv values of the baseline
absolute = ['Absolute'] * 31
relative = ['Relative'] * 31
baseline = ['Baseline'] *31
sketching = ['Sketching'] * 31
task_1_name = ['Task 1'] * 31
task_2_name = ['Task 2'] * 31
task_3_name = ['Task 3'] * 31

#--------------------------------------------------------------------------
#Task 1
#--------------------------------------------------------------------------
#Absolute
task_1_ab_baseline_data = np.column_stack((task_1_name,absolute,participant_list,task_1_rest_d, task_1_rest_t, task_1_rest_a, task_1_rest_b))
task_1_ab_baseline_values_df = pd.DataFrame(task_1_ab_baseline_data, columns=['Task','Power Type','Participant','Baseline delta power', ' Baseline theta power', 'Baseline alpha power', 'Baseline beta power'])
pd.options.display.float_format ='{:.2e}'.format

#Relative
# task_1_r_baseline_data = np.column_stack((task_1_name,relative,participant_list,rtask_1_rest_d, rtask_1_rest_t, rtask_1_rest_a, rtask_1_rest_b))
# task_1_r_baseline_values_df = pd.DataFrame(task_1_r_baseline_data, columns=['Task','Power Type','Participant','Baseline delta power', ' Baseline theta power', 'Baseline alpha power', 'Baseline beta power'])
# pd.options.display.float_format ='{:.2e}'.format

#--------------------------------------------------------------------------
#Task 2
#--------------------------------------------------------------------------
#Absolute
task_2_ab_baseline_data = np.column_stack((task_2_name,absolute,participant_list,task_2_rest_d, task_2_rest_t, task_2_rest_a, task_2_rest_b))
task_2_ab_baseline_values_df = pd.DataFrame(task_2_ab_baseline_data, columns=['Task','Power Type','Participant','Baseline delta power', ' Baseline theta power', 'Baseline alpha power', 'Baseline beta power'])
pd.options.display.float_format ='{:.2e}'.format

# #Relative
# task_2_r_baseline_data = np.column_stack((task_2_name,relative,participant_list,rtask_2_rest_d, rtask_2_rest_t, rtask_2_rest_a, rtask_2_rest_b))
# task_2_r_baseline_values_df = pd.DataFrame(task_2_r_baseline_data, columns=['Task','Power Type','Participant','Baseline delta power', ' Baseline theta power', 'Baseline alpha power', 'Baseline beta power'])
# pd.options.display.float_format ='{:.2e}'.format

#--------------------------------------------------------------------------
#Task 3
#--------------------------------------------------------------------------
#Absolute
task_3_ab_baseline_data = np.column_stack((task_3_name,absolute,participant_list,task_3_rest_d, task_3_rest_t, task_3_rest_a, task_3_rest_b))
task_3_ab_baseline_values_df = pd.DataFrame(task_3_ab_baseline_data, columns=['Task','Power Type','Participant','Baseline delta power', ' Baseline theta power', 'Baseline alpha power', 'Baseline beta power'])
pd.options.display.float_format ='{:.2e}'.format

#Relative
# task_3_r_baseline_data = np.column_stack((task_3_name,relative,participant_list,rtask_3_rest_d, rtask_3_rest_t, rtask_3_rest_a, rtask_3_rest_b))
# task_3_r_baseline_values_df = pd.DataFrame(task_3_r_baseline_data, columns=['Task','Power Type','Participant','Baseline delta power', ' Baseline theta power', 'Baseline alpha power', 'Baseline beta power'])
# pd.options.display.float_format ='{:.2e}'.format



In [25]:
bl_output_path ='/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/PSD Calculations/Full duration ABP/19th Nov/5s_Baseline ABP values.xlsx'
with pd.ExcelWriter(bl_output_path, engine='openpyxl') as writer:
    task_1_ab_baseline_values_df.to_excel(writer, sheet_name='task_1_ab', index=True)
    # # # task_1_r_baseline_values_df.to_excel(writer, sheet_name='task_1_r', index=True)
    task_2_ab_baseline_values_df.to_excel(writer, sheet_name='task_2_ab', index=True)
    # # task_2_r_baseline_values_df.to_excel(writer, sheet_name='task_2_r', index=True)
    task_3_ab_baseline_values_df.to_excel(writer, sheet_name='task_3_ab', index=True)
    # task_3_r_baseline_values_df.to_excel(writer, sheet_name='Stask_3_r', index=True)

#### Sketching signal abp calculation

In [26]:
# Cropping sketching signal from the full signal and storing in a participant-wise ist

i = 0
j = 0
k = 0

fif_1_sketching_signals = []
fif_2_sketching_signals = []
fif_3_sketching_signals = []

for fif in fif_task_1 :
    fif_temp = fif_task_1[i].copy()
    t1 = 30
    t2 = fif_temp.times[-1]
    fif_task_signal = fif_temp.crop(tmin=t1, tmax=t2)
    fif_1_sketching_signals.append(fif_task_signal)
    i = i+1

for fif in fif_task_2 :
    fif_temp = fif_task_2[j].copy()
    t1 = 30
    t2 = fif_temp.times[-1]
    fif_task_signal = fif_temp.crop(tmin=t1, tmax=t2)
    fif_2_sketching_signals.append(fif_task_signal)
    j = j+1

for fif in fif_task_3 :
    fif_temp = fif_task_3[k].copy()
    t1 = 30
    t2 = fif_temp.times[-1]
    fif_task_signal = fif_temp.crop(tmin=t1, tmax=t2)
    fif_3_sketching_signals.append(fif_task_signal)
    k = k+1
       

Cropping annotations 2023-11-03 14:49:16+00:00 - 2023-11-03 14:50:15+00:00
  [0] Keeping  (2023-11-03 14:49:21.580845+00:00 - 2023-11-03 14:49:22.329685+00:00 -> 35.580845 - 36.32968533203125)
  [1] Keeping  (2023-11-03 14:49:27.210556+00:00 - 2023-11-03 14:49:29.039692+00:00 -> 41.210556 - 43.03969189477539)
Cropping complete (kept 2)
Cropping annotations 2024-06-18 16:48:16+00:00 - 2024-06-18 16:49:39+00:00
  [0] Dropping (2024-06-18 16:47:47.539912+00:00 - 2024-06-18 16:47:47.917033+00:00: bad_signal)
  [1] Keeping  (2024-06-18 16:48:45.965961+00:00 - 2024-06-18 16:48:47.524730+00:00 -> 59.965961 - 61.52473022607422)
Cropping complete (kept 1)
Cropping annotations 2024-06-26 18:58:27+00:00 - 2024-06-26 18:59:11+00:00
  [0] Dropping (2024-06-26 18:58:25.988838+00:00 - 2024-06-26 18:58:26.768223+00:00: bad_signal)
  [1] Keeping  (2024-06-26 18:58:34.906918+00:00 - 2024-06-26 18:58:36.195416+00:00 -> 37.906918 - 39.195415924804685)
  [2] Keeping  (2024-06-26 18:58:42.008617+00:00 - 202

ABP calculations for sketching signal

In [30]:
%%time
i = 0
j = 0
k = 0
window_length_fs = 5
reject_criteria = dict(eeg=200e-6)


task1_ab_mean_delta_abp = np.zeros((len(fif_1_sketching_signals)))
task1_ab_mean_theta_abp = np.zeros((len(fif_1_sketching_signals)))
task1_ab_mean_alpha_abp = np.zeros((len(fif_1_sketching_signals)))
task1_ab_mean_beta_abp = np.zeros((len(fif_1_sketching_signals))) 

# task1_r_mean_delta_abp = np.zeros((len(fif_1_sketching_signals)))
# task1_r_mean_theta_abp = np.zeros((len(fif_1_sketching_signals)))
# task1_r_mean_alpha_abp = np.zeros((len(fif_1_sketching_signals)))
# task1_r_mean_beta_abp = np.zeros((len(fif_1_sketching_signals)))

task2_ab_mean_delta_abp = np.zeros((len(fif_2_sketching_signals)))
task2_ab_mean_theta_abp = np.zeros((len(fif_2_sketching_signals)))
task2_ab_mean_alpha_abp = np.zeros((len(fif_2_sketching_signals)))
task2_ab_mean_beta_abp = np.zeros((len(fif_2_sketching_signals))) 

# task2_r_mean_delta_abp = np.zeros((len(fif_2_sketching_signals)))
# task2_r_mean_theta_abp = np.zeros((len(fif_2_sketching_signals)))
# task2_r_mean_alpha_abp = np.zeros((len(fif_2_sketching_signals)))
# task2_r_mean_beta_abp = np.zeros((len(fif_2_sketching_signals)))

task3_ab_mean_delta_abp = np.zeros((len(fif_3_sketching_signals)))
task3_ab_mean_theta_abp = np.zeros((len(fif_3_sketching_signals)))
task3_ab_mean_alpha_abp = np.zeros((len(fif_3_sketching_signals)))
task3_ab_mean_beta_abp = np.zeros((len(fif_3_sketching_signals))) 

# task3_r_mean_delta_abp = np.zeros((len(fif_3_sketching_signals)))
# task3_r_mean_theta_abp = np.zeros((len(fif_3_sketching_signals)))
# task3_r_mean_alpha_abp = np.zeros((len(fif_3_sketching_signals)))
# task3_r_mean_beta_abp = np.zeros((len(fif_3_sketching_signals)))


#Task 1 
for item in fif_1_sketching_signals :
    temp_fif = fif_1_sketching_signals[i].copy()
    x = temp_fif.times[0]
    y = temp_fif.times[-1]
    equal_event = mne.make_fixed_length_events(temp_fif, start=0, stop=y, duration=window_length_fs,first_samp=True)
    # print('--------------------------------------')
    # print('--------------------------------------')
    # print(equal_event)
    # print('--------------------------------------')
    # print('--------------------------------------')
    temp_epoch = mne.Epochs(temp_fif, equal_event, tmin=0, tmax=window_length_fs ,baseline=None, detrend=1, reject = reject_criteria, reject_by_annotation= True, preload=True)
    temp_dp,temp_tp,temp_ap,temp_bp = average_bandpowers(temp_epoch)
    task1_ab_mean_delta_abp[i] = temp_dp
    task1_ab_mean_theta_abp[i] = temp_tp
    task1_ab_mean_alpha_abp[i] = temp_ap
    task1_ab_mean_beta_abp[i] = temp_bp
    # rtemp_dp,rtemp_tp,rtemp_ap,rtemp_bp = average_bandpowers_relative(temp_epoch)
    # task1_r_mean_delta_abp[i] = rtemp_dp
    # task1_r_mean_theta_abp[i] = rtemp_tp
    # task1_r_mean_alpha_abp[i] = rtemp_ap
    # task1_r_mean_beta_abp[i] = rtemp_bp
    print(f' Stored absolute and relative abp values for participant {i+1}')
    i = i+1

#Task 2 
for item in fif_2_sketching_signals :
    temp_fif = fif_2_sketching_signals[j].copy()
    x = temp_fif.times[0]
    y = temp_fif.times[-1]
    equal_event = mne.make_fixed_length_events(temp_fif, start=0, stop=y, duration=window_length_fs,first_samp=True)
    # print('--------------------------------------')
    # print('--------------------------------------')
    # print(equal_event)
    # print('--------------------------------------')
    # print('--------------------------------------')
    temp_epoch = mne.Epochs(temp_fif, equal_event, tmin=0, tmax=window_length_fs ,baseline=None, detrend=1, reject = reject_criteria, reject_by_annotation= True, preload=True)
    temp_dp,temp_tp,temp_ap,temp_bp = average_bandpowers(temp_epoch)
    task2_ab_mean_delta_abp[j] = temp_dp
    task2_ab_mean_theta_abp[j] = temp_tp
    task2_ab_mean_alpha_abp[j] = temp_ap
    task2_ab_mean_beta_abp[j] = temp_bp
    # rtemp_dp,rtemp_tp,rtemp_ap,rtemp_bp = average_bandpowers_relative(temp_epoch)
    # task2_r_mean_delta_abp[j] = rtemp_dp
    # task2_r_mean_theta_abp[j] = rtemp_tp
    # task2_r_mean_alpha_abp[j] = rtemp_ap
    # task2_r_mean_beta_abp[j] = rtemp_bp
    print(f' Stored absolute and relative abp values for participant {j+1}')
    j = j+1

#Task 3 
for item in fif_3_sketching_signals :
    temp_fif = fif_3_sketching_signals[k].copy()
    x = temp_fif.times[0]
    y = temp_fif.times[-1]
    equal_event = mne.make_fixed_length_events(temp_fif, start=0, stop=y, duration=window_length_fs,first_samp=True)
    # print('--------------------------------------')
    # print('--------------------------------------')
    # print(equal_event)
    # print('--------------------------------------')
    # print('--------------------------------------')
    temp_epoch = mne.Epochs(temp_fif, equal_event, tmin=0, tmax=window_length_fs ,baseline=None, detrend=1, reject = reject_criteria, reject_by_annotation= True, preload=True)
    temp_dp,temp_tp,temp_ap,temp_bp = average_bandpowers(temp_epoch)
    task3_ab_mean_delta_abp[k] = temp_dp
    task3_ab_mean_theta_abp[k] = temp_tp
    task3_ab_mean_alpha_abp[k] = temp_ap
    task3_ab_mean_beta_abp[k] = temp_bp
    # rtemp_dp,rtemp_tp,rtemp_ap,rtemp_bp = average_bandpowers_relative(temp_epoch)
    # task3_r_mean_delta_abp[k] = rtemp_dp
    # task3_r_mean_theta_abp[k] = rtemp_tp
    # task3_r_mean_alpha_abp[k] = rtemp_ap
    # task3_r_mean_beta_abp[k] = rtemp_bp
    print(f' Stored absolute and relative abp values for participant {k+1}')
    k = k+1


Not setting metadata
13 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 13 events and 1281 original time points ...
    Getting epoch for 0-1281
    Getting epoch for 1280-2561
    Getting epoch for 2560-3841
    Getting epoch for 3840-5121
    Getting epoch for 5120-6401
    Getting epoch for 6400-7681
    Getting epoch for 7680-8961
    Getting epoch for 8960-10241
    Getting epoch for 10240-11521
    Getting epoch for 11520-12801
    Getting epoch for 12800-14081
    Getting epoch for 14080-15361
    Getting epoch for 15360-16641
0 bad epochs dropped
Data is self data: False
Data is self data: True


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 3.011290657728456e-10
Final Mean Theta Power across epochs: 1.537219759250221e-10
Final Mean Alpha Power across epochs: 3.301854700929231e-10
Final Mean Beta Power across epochs: 1.5150517562779975e-10
 Stored absolute and relative abp values for participant 1
Not setting metadata
11 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11 events and 1281 original time points ...
    Getting epoch for 0-1281
    Getting epoch for 1280-2561
    Getting epoch for 2560-3841
    Getting epoch for 3840-5121
    Getting epoch for 5120-6401
    Getting epoch for 6400-7681
    Getting epoch for 7680-8961
    Getting epoch for 8960-10241
    Getting epoch for 10240-11521
    Getting epoch for 11520-12801
    Getting epoch for 12800-14081
2 bad epochs dropped
Data is self data: False
Data is self data: True


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 2.394305401886708e-10
Final Mean Theta Power across epochs: 1.4038981973765216e-10
Final Mean Alpha Power across epochs: 1.4342263238326475e-10
Final Mean Beta Power across epochs: 2.34300274970974e-10
 Stored absolute and relative abp values for participant 2
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 1281 original time points ...
    Getting epoch for 0-1281
    Getting epoch for 1280-2561
    Getting epoch for 2560-3841
    Getting epoch for 3840-5121
    Getting epoch for 5120-6401
    Getting epoch for 6400-7681
    Getting epoch for 7680-8961
    Getting epoch for 8960-10241
    Getting epoch for 10240-11521
    Getting epoch for 11520-12801
0 bad epochs dropped
Data is self data: False
Data is self data: True


...

Final Mean Delta Power across epochs: 2.2984633839035863e-10
Final Mean Theta Power across epochs: 8.86586832278254e-11
Final Mean Alpha Power across epochs: 7.085140603168304e-11
Final Mean Beta Power across epochs: 1.1768559377354448e-10
 Stored absolute and relative abp values for participant 29
Not setting metadata
63 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 63 events and 1281 original time points ...
    Getting epoch for 0-1281
    Getting epoch for 1280-2561
    Getting epoch for 2560-3841
    Getting epoch for 3840-5121
    Getting epoch for 5120-6401
    Getting epoch for 6400-7681
    Getting epoch for 7680-8961
    Getting epoch for 8960-10241
    Getting epoch for 10240-11521
    Getting epoch for 11520-12801
    Getting epoch for 12800-14081
    Getting epoch for 14080-15361
    Getting epoch for 15360-16641
    Getting epoch for 16640-17921
    Getting epoch for 17920-19201
    Getting epoch for 19

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 1.5179407521961977e-10
Final Mean Theta Power across epochs: 8.919510411947403e-11
Final Mean Alpha Power across epochs: 7.661874479328251e-11
Final Mean Beta Power across epochs: 1.543006353248413e-10
 Stored absolute and relative abp values for participant 30
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1281 original time points ...
    Getting epoch for 0-1281
    Getting epoch for 1280-2561
    Getting epoch for 2560-3841
    Getting epoch for 3840-5121
    Getting epoch for 5120-6401
    Getting epoch for 6400-7681
    Getting epoch for 7680-8961
    Getting epoch for 8960-10241
    Getting epoch for 10240-11521
    Getting epoch for 11520-12801
    Getting epoch for 12800-14081
    Getting epoch for 14080-15361
    Getting epoch for 15360-16641
    Getting epoch for 16640-17921
    Getting epoch for 17920-19201
    Getting epoch for 19

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_76615/4038551095.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs_8.get_data()


Final Mean Delta Power across epochs: 8.080747308679509e-11
Final Mean Theta Power across epochs: 4.163984425465298e-11
Final Mean Alpha Power across epochs: 2.452337567841956e-11
Final Mean Beta Power across epochs: 9.976233223930413e-11
 Stored absolute and relative abp values for participant 31
CPU times: user 1h 52min 20s, sys: 15min 5s, total: 2h 7min 25s
Wall time: 24min 9s


In [31]:
#Creating DF for all bandpower calculations
#--------------------------------------------------------------------------
#Task 1
#--------------------------------------------------------------------------
#Absolute
task_1_ab_abp_data = np.column_stack((task_1_name,absolute,participant_list,task1_ab_mean_delta_abp, task1_ab_mean_theta_abp, task1_ab_mean_alpha_abp, task1_ab_mean_beta_abp))
task_1_ab_abp_df = pd.DataFrame(task_1_ab_abp_data, columns=['Task','Power Type','Participant','Mean delta power', ' Mean theta power', 'Mean alpha power', 'Mean beta power'])
pd.options.display.float_format ='{:.2e}'.format

#Relative
# task_1_r_abp_data = np.column_stack((task_1_name,relative,participant_list,task1_r_mean_delta_abp, task1_r_mean_theta_abp, task1_r_mean_alpha_abp, task1_r_mean_beta_abp))
# task_1_r_abp_df = pd.DataFrame(task_1_r_abp_data, columns=['Task','Power Type','Participant','Mean delta power', ' Mean theta power', 'Mean alpha power', 'Mean beta power'])
# pd.options.display.float_format ='{:.2e}'.format

#--------------------------------------------------------------------------
#Task 2
#--------------------------------------------------------------------------
#Absolute
task_2_ab_abp_data = np.column_stack((task_2_name,absolute,participant_list,task2_ab_mean_delta_abp, task2_ab_mean_theta_abp, task2_ab_mean_alpha_abp, task2_ab_mean_beta_abp))
task_2_ab_abp_df = pd.DataFrame(task_2_ab_abp_data, columns=['Task','Power Type','Participant','Mean delta power', ' Mean theta power', 'Mean alpha power', 'Mean beta power'])
pd.options.display.float_format ='{:.2e}'.format

#Relative
# task_2_r_abp_data = np.column_stack((task_2_name,relative,participant_list,task2_r_mean_delta_abp, task2_r_mean_theta_abp, task2_r_mean_alpha_abp, task2_r_mean_beta_abp))
# task_2_r_abp_df = pd.DataFrame(task_2_r_abp_data, columns=['Task','Power Type','Participant','Mean delta power', ' Mean theta power', 'Mean alpha power', 'Mean beta power'])
# pd.options.display.float_format ='{:.2e}'.format

#--------------------------------------------------------------------------
#Task 3
#--------------------------------------------------------------------------
#Absolute
task_3_ab_abp_data = np.column_stack((task_3_name,absolute,participant_list,task3_ab_mean_delta_abp, task3_ab_mean_theta_abp, task3_ab_mean_alpha_abp, task3_ab_mean_beta_abp))
task_3_ab_abp_df = pd.DataFrame(task_3_ab_abp_data, columns=['Task','Power Type','Participant','Mean delta power', ' Mean theta power', 'Mean alpha power', 'Mean beta power'])
pd.options.display.float_format ='{:.2e}'.format

#Relative
# task_3_r_abp_data = np.column_stack((task_3_name,relative,participant_list,task3_r_mean_delta_abp, task3_r_mean_theta_abp, task3_r_mean_alpha_abp, task3_r_mean_beta_abp))
# task_3_r_abp_df = pd.DataFrame(task_3_r_abp_data, columns=['Task','Power Type','Participant','Mean delta power', ' Mean theta power', 'Mean alpha power', 'Mean beta power'])
# pd.options.display.float_format ='{:.2e}'.format


In [32]:
bl_output_path ='/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/PSD Calculations/Full duration ABP/19th Nov/5s_Task_1_2_3 ABP values.xlsx'
with pd.ExcelWriter(bl_output_path, engine='openpyxl') as writer:
    task_1_ab_abp_df.to_excel(writer, sheet_name='task_1_ab', index=True)
    # task_1_r_abp_df.to_excel(writer, sheet_name='task_1_r', index=True)
    task_2_ab_abp_df.to_excel(writer, sheet_name='task_2_ab', index=True)
    # task_2_r_abp_df.to_excel(writer, sheet_name='task_2_r', index=True)
    task_3_ab_abp_df.to_excel(writer, sheet_name='task_3_ab', index=True)
    # task_3_r_abp_df.to_excel(writer, sheet_name='task_3_r', index=True)

### Total power calculations

Sketching signals are stored in the lists below for all three tasks:
fif_1_sketching_signals 
fif_2_sketching_signals 
fif_3_sketching_signals 

In [ ]:
total_power_array_t1 = [0] * len(fif_1_sketching_signals)
total_power_array_t2 = [0] * len(fif_2_sketching_signals)
total_power_array_t3 = [0] * len(fif_3_sketching_signals)
i = 0
j = 0
k = 0

for fif in fif_1_sketching_signals:
    total_power_array_t1[i] = sum_total_channel_power(fif)
    i = i+1

for fif in fif_2_sketching_signals:
    total_power_array_t2[j] = sum_total_channel_power(fif)
    j = j+1

for fif in fif_3_sketching_signals:
    total_power_array_t3[k] = sum_total_channel_power(fif)
    k = k+1

In [ ]:
#Total power df
taskwise_tp_data = np.column_stack((participant_list,total_power_array_t1,total_power_array_t2,total_power_array_t3))
taskwise_tp_df = pd.DataFrame(taskwise_tp_data, columns=['Participant','Task 1 total power', ' Task 2 total power', 'Task 3 total power'])
pd.options.display.float_format ='{:.2e}'.format

In [ ]:
taskwise_tp_df

In [ ]:
taskwise_tp_df.to_csv('/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/PSD Calculations/Full duration ABP/7th Nov/Taskwise total powers.csv', index=True)